Lab02
======

## 회귀 분석 연습문제
이번 실습 수업에 배운 회귀 분석 방법을 사용하여 Boston Housing, 보스턴 주택 가격 데이터를 회귀 분석 해보겠습니다.<br>
이전 챕터의 예제 코드를 기반으로 회귀분석을 진행하시면 됩니다.<br>
주택 가격을 Target으로 하여 이에 영향을 미치는 독립변수를 찾아 설명력이 높은 회귀식을 추정해 보세요.

In [ ]:
from os.path import join
import numpy as np

from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score
from sklearn.tree import export_graphviz

import matplotlib.pyplot as plt

import statsmodels.api as sm

import pandas as pd
from pandas.plotting import scatter_matrix

## 1. 데이터 살펴보기

이번 실습에 사용할 데이터는보스턴 시의 주택 가격에 대한 데이터입니다.

주택의 여러가지 요건들과 주택의 가격 정보가 포함되어 있는 데이터 셋을 분석하여, 주택의 가격에 영향을 미치는 요소를 분석해보세요.

### Feature Description

1. CRIM : 자치시(town) 별 1인당 범죄율
2. ZN : 25,000 평방피트를 초과하는 거주지역의 비율
3. INDUS : 비소매상업지역 면적 비율

5. NOX : 일산화질소 농도
6. RM : 주택 1가구당 평균 방의 개수
7. AGE : 1940년 이전에 건축된 소유주택의 비율
8. DIS : 5개의 보스턴 직업센터까지의 접근성 지수
9. RAD : 방사형 도로까지의 접근성 지수
10. TAX : 10,000 달러 당 재산세율
11. PTRATIO : 자치시(town)별 학생/교사 비율
12. B : 1000(Bk-0.63)^2, 여기서 Bk는 자치시별 흑인의 비율 
13. LSTAT : 모집단의 하위계층의 비율(%)

14. MEDV : 본인 소유의 주택가격(중앙값) (단위: $1,000)

15. CAT.MEDV : if MEDV > 30, CAT.MEDV = 1 else, 0

데이터를 살펴보기 쉽게 csv 파일을 pandas DataFrame으로 읽습니다.

In [ ]:
df = pd.read_csv(join('data', 'BostonHousing.csv'))

DataFrame.head() 함수로 처음 5개의 데이터를 살펴볼 수 있습니다.

In [ ]:
df.head()

## 2. 간단한 전처리

In [ ]:
for dim in df.columns:
        df[dim] -= np.min(df[dim])
        df[dim] /= np.max(df[dim])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

### find relationship
최적의 모델 파라미터를 추정해보세요.

In [ ]:
df.corr()

In [ ]:
dfx = df.iloc[:,:-2]
dfy = df.iloc[:,-2]

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in= 0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(dfx, dfy)

print('resulting features:')
print(result)

newlabel = result

In [ ]:
dfx_multi = df[newlabel]
dfy_multi = df[['MEDV']]

# 선형회귀 추정기 생성
lr = linear_model.LinearRegression() 

# input 대해 선형 회귀(모델 파라미터 추정)
lr.fit(dfx_multi.values.tolist(), dfy_multi.values.tolist())

# 회귀식으로 데이터의 결과 추정
dfy_y_pred_multi = lr.predict(dfx_multi.values.tolist())

# R2 scroe
R2 = r2_score(dfy_multi.values.tolist(), dfy_y_pred_multi)
print('R2 score: %.2f' % R2)

# adj R2 score
n = len(dfx_multi)
p = len(newlabel)

Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
print('Adj R2 score: %.2f' % Adj_r2)

In [ ]:
# 모델 파라미터 출력
print('Model parameters: \n')
rgstr = ''
b = []
for i in range(len(lr.coef_[0])):
    b.append(lr.coef_[0,i])
    print("b%d" %(len(lr.coef_[0])-i),": ", "%f" %lr.coef_[0,i])
    rgstr = ' + '+repr(lr.coef_[0,i]) + '*x'+repr(len(lr.coef_[0])-i) + rgstr

print('b0:', lr.intercept_[0])
print()
rgstr = repr(lr.intercept_[0]) + rgstr
print('y = ', rgstr)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(dfy_multi.values.tolist(),dfy_y_pred_multi)

In [ ]:
# X ,Y 데이터 나누기.
label = ['ZN']
dfx = df[label]
dfy = df[['MEDV']]

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(dfx, dfy)

In [ ]:
y_pred = lr.predict(dfx)

In [ ]:
print(lr.coef_)
print(lr.intercept_)

In [ ]:
print(lr.score(dfx, dfy))

In [ ]:
# # 모델 파라미터 출력
# print('Model parameters: \n')

# rgstr = ''
# for i in range(len(lr.coef_[0])):
#     print("b%d" %(len(lr.coef_[0])-i),": ", "%f" %lr.coef_[0,i])
#     rgstr = ' + '+repr(lr.coef_[0,i]) + '*x'+repr(len(lr.coef_[0])-i) + rgstr

# print('b0:', lr.intercept_[0])
# print()
# rgstr = repr(lr.intercept_[0]) + rgstr
# print('y = ', rgstr)

### evaluation
R2를 이용하여 테스트 결과를 평가합니다.